In [31]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [32]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [33]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')

# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
# Set all non-zero amounts to 1
pivoted_data = pivoted_data.applymap(lambda x: 1 if x > 0 else 0)

pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,2017-09-06,1,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
97,7.50,36.1,2017-09-07,1,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
190,5.81,36.4,2017-09-08,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
283,7.50,36.6,2017-09-09,1,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
376,7.50,35.9,2017-09-10,1,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19348,5.37,36.1,2018-04-02,1,1,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
19441,5.82,36.5,2018-04-03,1,1,0,1,0,0,1,...,0,0,0,0,0,1,1,0,0,0
19534,5.03,36.1,2018-04-04,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
19627,6.00,36.3,2018-04-05,1,0,0,0,0,0,1,...,0,1,0,1,0,0,1,0,0,0


In [34]:
from sklearn.model_selection import train_test_split

X= joined_data.iloc[:,0:1]
X

,PH
0,5.40
97,7.50
190,5.81
283,7.50
376,7.50
...,...
19348,5.37
19441,5.82
19534,5.03
19627,6.00


In [35]:
y = joined_data.iloc[:,5:]
y

,Apple Pie,Banana,Beer,Braised Potatoes,Bread,Buckwheat porridge,Bun,Cake,Candies,Cheese,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
97,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
190,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
283,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
376,0,1,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19348,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
19441,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
19534,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
19627,0,0,0,0,1,0,0,1,0,1,...,0,1,0,1,0,0,1,0,0,0


In [36]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        if amount != 0:
            amount = 1
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

                                     food_and_amount
0  [(Gym, 0.0), (Liquid, 1), (Lemon water, 1), (T...
1  [(Gym, 1), (Liquid, 1), (Lemon water, 1), (Tea...
2  [(Gym, 0.0), (Liquid, 1), (Lemon water, 1), (T...
3  [(Gym, 0.0), (Liquid, 1), (Lemon water, 1), (T...
4  [(Gym, 1), (Liquid, 1), (Lemon water, 1), (Tea...


In [37]:
df = pd.DataFrame(y)
df2 = pd.DataFrame(X)
# convert column 'A' to a tuple
food_and_amount = tuple(df['food_and_amount'].tolist())
ph = tuple(df2['PH'].tolist())
print(ph)

(5.4, 7.5, 5.81, 7.5, 7.5, 5.67, 7.5, 5.81, 6.0, 7.5, 7.5, 5.94, 7.5, 7.5, 7.5, 7.5, 7.5, 5.48, 7.5, 5.6, 7.5, 7.5, 7.5, 5.87, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 5.97, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 5.99, 5.63, 5.65, 7.5, 5.97, 4.98, 5.14, 7.5, 7.5, 5.35, 7.5, 5.86, 5.69, 5.31, 5.43, 5.12, 5.85, 5.95, 7.5, 5.96, 5.64, 7.5, 5.68, 5.73, 7.5, 5.97, 5.94, 5.99, 7.5, 5.52, 5.68, 7.5, 5.88, 7.5, 5.47, 5.72, 4.96, 5.67, 7.5, 7.5, 7.5, 5.95, 7.5, 7.5, 5.95, 7.5, 5.51, 5.38, 5.34, 5.71, 7.5, 5.57, 5.02, 5.47, 5.24, 7.5, 5.57, 7.5, 5.79, 5.5, 7.5, 5.99, 5.56, 5.38, 5.59, 5.46, 7.5, 5.07, 5.77, 5.87, 7.5, 7.5, 5.54, 5.89, 5.6, 5.97, 7.5, 7.5, 7.5, 7.5, 7.5, 5.77, 7.5, 7.5, 7.5, 7.5, 5.99, 7.5, 5.98, 7.5, 5.81, 7.5, 7.5, 7.5, 7.5, 7.5, 5.81, 5.77, 7.5, 7.5, 7.5, 6.0, 5.86, 7.5, 5.89, 7.5, 5.84, 7.5, 7.5, 5.74, 5.69, 5.35, 5.59, 5.77, 5.54, 5.39, 5.33, 5.52, 7.5

In [41]:
food_and_amount

([('Gym', 0.0),
  ('Liquid', 1),
  ('Lemon water', 1),
  ('Tea(black/green)', 0.0),
  ('Fruit tea', 1),
  ('Water', 1),
  ('Latte', 0.0),
  ('Mineral water (Esentuki - 4)', 0.0),
  ('Mineral water (Borjomi)', 0.0),
  ('Mineral water (Prolom)', 0.0),
  ('Tequila', 0.0),
  ('Red wine', 1),
  ('White wine', 0.0),
  ('Strong alcohol', 0.0),
  ('Beer', 0.0),
  ('Morse', 0.0),
  ('Milk', 0.0),
  ('Orange juice', 0.0),
  ('Oatmeal', 1),
  ('Pizza with meat', 0.0),
  ('Cheese Pizza', 0.0),
  ('Cheesecakes', 0.0),
  ('Scrambled eggs', 0.0),
  ('Mayonnaise salad', 0.0),
  ('Bread', 0.0),
  ('Meat', 0.0),
  ('Sauce', 0.0),
  ('Cutlet', 0.0),
  ('Sausage', 0.0),
  ('Fish', 0.0),
  ('Sushi', 0.0),
  ('Cod liver', 0.0),
  ('Cheese', 0.0),
  ('Curd', 0.0),
  ('Sour cream', 0.0),
  ('Buckwheat porridge', 0.0),
  ('Noodles with vegetables', 0.0),
  ('Noodles with shrimp', 0.0),
  ('Meat pilaf', 0.0),
  ('Meatless pilaf', 0.0),
  ('Pea soup with meat', 0.0),
  ('Fried vegetables', 0.0),
  ('Pyagse with 

In [43]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

data = pd.DataFrame({
    'pH_level': ph,
    'food_and_activities': food_and_amount
})

def categorize_pH(pH):
    if pH < 7.0:
        return "acidic"
    elif pH > 7.0:
        return "alkaline"
    else:
        return "neutral"

data['pH_category'] = data['pH_level'].apply(categorize_pH)

def binary_representation(row):
    binary_list = [row['pH_category']]
    for food, amount in row['food_and_activities']:
        if amount > 0:
            binary_list.append(food)
    return binary_list

data['binary'] = data.apply(binary_representation, axis=1)

transactions = data['binary'].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
binary_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(binary_df, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Filter the rules based on the antecedent (left-hand side) containing the pH category "acidic"
acidic_rules = rules[rules['antecedents'].apply(lambda x: 'acidic' in x)]
alkaline_rules = rules[rules['antecedents'].apply(lambda x: 'alkaline' in x)]

print(alkaline_rules)


                   antecedents  \
53                  (alkaline)   
87                  (alkaline)   
140                 (alkaline)   
259                 (alkaline)   
326                 (alkaline)   
...                        ...   
1281679      (alkaline, Water)   
1281682  (alkaline, Fruit tea)   
1281684    (Oatmeal, alkaline)   
1281686   (alkaline, Red wine)   
1281694             (alkaline)   

                                               consequents  \
53                                                 (Apple)   
87                                                (Banana)   
140                                                (Bread)   
259                                               (Cookie)   
326                                       (Dried apricots)   
...                                                    ...   
1281679  (The nuts, Liquid, Mineral water (Esentuki - 4...   
1281682  (The nuts, Liquid, Mineral water (Esentuki - 4...   
1281684  (The nuts, Liquid, Miner

In [44]:
def recommend_food_and_activities(pH_level, rules):
    pH_category = categorize_pH(pH_level)

    # Filter the rules based on the antecedent (left-hand side) containing the pH category
    recommendations = rules[rules['antecedents'].apply(lambda x: pH_category in x)]

    # Sort the rules by confidence and select the top 5 rules
    top_recommendations = recommendations.sort_values(by='confidence', ascending=False).head(5)

    # Extract the consequents (right-hand side) of the rules
    recommended_items = set()
    for _, row in top_recommendations.iterrows():
        recommended_items.update(row['consequents'])

    # Remove the pH category from the recommended items
    recommended_items.discard(pH_category)

    return list(recommended_items)


In [45]:
# Example usage
urine_pH = 9 # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, alkaline_rules)
print("Recommended food and activities:", recommended_items)

Recommended food and activities: ['Liquid', 'Lemon water', 'Fruit tea']


In [46]:
import joblib

# Save the association rules DataFrame to a file
joblib.dump(alkaline_rules, 'alkaline_rules.joblib')

['alkaline_rules.joblib']